In [ ]:
import os; os.environ["CONDA_PREFIX"] = "/opt/conda"
!pip install -U -q uv

In [ ]:
!uv pip install modAL-python torchvision torchmetrics mlxtend torchsummary scikit-learn-intelex

# Imports

In [ ]:
import random
from pathlib import Path
from timeit import default_timer as timer
import matplotlib.pyplot as plt
import mlxtend
import numpy as np
import requests
import torch
import torchvision
import torchvision.models as models
from mlxtend.plotting import plot_confusion_matrix
from modAL.models import ActiveLearner
from modAL.uncertainty import entropy_sampling, margin_sampling, uncertainty_sampling
from skorch import NeuralNetClassifier
from torch import nn
from torch.utils.data import DataLoader
from torchmetrics import ConfusionMatrix, F1Score, Precision, Recall
from torchsummary import summary
from torchvision import datasets
from torchvision.transforms import ToTensor
from tqdm.auto import tqdm
from sklearnex import patch_sklearn
import warnings
from sklearn.linear_model import LogisticRegression
import tensorflow_hub as hub
from datasets import load_dataset
import pandas as pd

os.mkdir('reports')
patch_sklearn()
print(f"mlxtend version: {mlxtend.__version__}")
print("torch", torch.__version__)
print("torchvision", torchvision.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU List:", torch.cuda.device_count())
print("Current Device:", torch.cuda.current_device())
warnings.filterwarnings("ignore")

# Cifar 10


In [ ]:
cifar_train_data = datasets.CIFAR10(root="data", train=True, download=True, transform=ToTensor(), target_transform=None)
cifar_test_data = datasets.CIFAR10(root="data", train=False, download=True, transform=ToTensor(), target_transform=None)

x_train, y_train = cifar_train_data.data, np.array(cifar_train_data.targets)
x_test, y_test = cifar_test_data.data, np.array(cifar_test_data.targets)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
cifar_class_names = cifar_train_data.classes
print(cifar_class_names)

In [ ]:
num_classes = len(cifar_class_names)

In [ ]:
class_to_idx_cif = cifar_train_data.class_to_idx
print(class_to_idx_cif)

In [ ]:
# Show more images
plt.figure(figsize=(9, 9))
rows, cols = 4, 4

for i in range(1, rows * cols + 1):
    random_index = torch.randint(1, len(cifar_train_data), size=[1]).item()
    plt.subplot(rows, cols, i)
    image, label = cifar_train_data[random_index]
    image = image.permute(1, 2, 0).numpy()
    plt.imshow(image, interpolation="bilinear")
    plt.title(cifar_class_names[label])
    plt.axis(False)

plt.show()
plt.savefig('reports/cifar_imgs_labeled.png')

# Unlabeling The Dataset

In [ ]:
# Define the percentage of data without labels
percentage_without_labels = 0.8

# Calculate the number of samples without labels
num_samples_without_labels = int(len(cifar_train_data) * percentage_without_labels)
num_samples_with_labels = len(cifar_train_data) - num_samples_without_labels

# Create indices for data with labels and without labels
indices_without_labels = np.random.choice(range(len(cifar_train_data)), size=num_samples_without_labels, replace=False)
indices_with_labels = np.array([i for i in range(len(cifar_train_data)) if i not in indices_without_labels])

x_initial = x_train[indices_with_labels]
y_initial = y_train[indices_with_labels]

# generate the pool
# remove the initial data from the training dataset
x_pool = np.delete(x_train, indices_with_labels, axis=0)
y_pool = np.delete(y_train, indices_with_labels, axis=0)

In [ ]:
x_initial.shape, y_initial.shape, x_pool.shape, y_pool.shape

In [ ]:
len(cifar_train_data)

In [ ]:
cifar_train_data[1][1]

# Creating the DataLoaders


In [ ]:
def numpy_to_dataloader(x, y, batch_size=32):
    dataset = torch.utils.data.TensorDataset(torch.tensor(x, dtype=torch.float32).permute(0, 3, 1, 2), torch.tensor(y))
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
BATCH_SIZE_CIF = 32

cifar_train_data_loader = numpy_to_dataloader(x_initial, y_initial, BATCH_SIZE_CIF)
cifar_test_data_loader = numpy_to_dataloader(x_test, y_test, BATCH_SIZE_CIF)

In [ ]:
cifar_train_data_loader, cifar_test_data_loader

In [ ]:
print(f"Length of Training Data loader: {len(cifar_train_data_loader)}, Batches of {cifar_train_data_loader.batch_size}")
print(f"Length of Testing Data loader: {len(cifar_test_data_loader)}, Batches of {cifar_test_data_loader.batch_size}")

In [ ]:
train_features_batch_cif, train_labels_batch_cif = next(iter(cifar_train_data_loader))
(
    train_features_batch_cif.shape,
    train_labels_batch_cif.shape,
)  # [Batch_Size, Color_Channels, Height, Width] Color Channels First!

# Importing and Using ResNet 50 Architecture


In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

In [ ]:
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 10)
next(model.parameters()).device

In [ ]:
f = nn.Flatten(start_dim=0)  # The default start dim is 1
x = f(torch.randn(10, 7, 7))
x.size()

In [ ]:
rand_image_tensor = torch.randn(size=(32, 3, 32, 32))

In [ ]:
rand_image_tensor.device

In [ ]:
model(rand_image_tensor)

In [ ]:
model.to(device)
next(model.parameters()).device

# Printing the Architecture and Number of Trianing Parameters in each Layer


In [ ]:
summary(model, (3, 32, 32))

# External Helper Functions


In [ ]:
if Path("HelperFunctions.py").is_file():
    print("Helper Functions already exists, skipping downloading")
else:
    print("downloading HelperFunctions.py")
    request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
    with open("HelperFunctions.py", "wb") as f:
        f.write(request.content)

In [ ]:
# Picking a loss function and an optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)

In [ ]:
def accuracy_fn(y_pred, y_true):
    correct = torch.eq(y_pred, y_true).sum().item()
    accuracy = correct / len(y_true) * 100
    return accuracy

In [ ]:
def display_train_time(start: float, end: float, device: torch.device = None):
    total_time = end - start
    print(f"Train time on device {device}: {total_time:.3f} seconds")
    return total_time

In [ ]:
start = timer()
end = timer()
display_train_time(start, end, device="cpu")

In [ ]:
def train_step(
    model: nn.Module,
    data_loader: torch.utils.data.dataloader,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    accuracy_fn,
    device: torch.device = device,
):
    training_loss = 0
    training_acc = 0
    model.train()

    for batch, (x, y) in enumerate(data_loader):
        # Put the data on the target device
        x, y = x.to(device), y.to(device)
        # Forward Pass
        y_pred = model(x)
        # Loss
        loss = loss_fn(y_pred, y)
        training_loss += loss
        training_acc += accuracy_fn(y_pred.argmax(dim=1), y)
        # Optimizer zero grad
        optimizer.zero_grad()
        # Loss Backward
        loss.backward()
        # optimizer step step step
        optimizer.step()
    # After looping over batches, divide the training loss over the number of batches to get the average loss per batch
    training_loss /= len(
        data_loader
    )  # train.data_loader (returns the number of batches)
    training_acc /= len(data_loader)
    print(f"Training Loss: {training_loss:.5f} | Training Accuracy: {training_acc:.2f}%")
    return training_acc, training_loss

In [ ]:
def test_step(
    model: nn.Module,
    data_loader: torch.utils.data.dataloader,
    loss_fn: nn.Module,
    accuracy_fn,
    device: torch.device = device,
):
    testing_acc = 0
    testing_loss = 0
    model.eval()

    with torch.inference_mode():
        for x, y in data_loader:
            x, y = x.to(device), y.to(device)
            # Forward Pass
            y_test_pred = model(x)
            # Loss
            testing_loss += loss_fn(y_test_pred, y)
            # Accuracy
            testing_acc += accuracy_fn(y_test_pred.argmax(dim=1), y)
        testing_loss /= len(data_loader)
        testing_acc /= len(data_loader)
    print(f"Testing Loss: {testing_loss:.3f} | Testing Accuracy: {testing_acc:.2f}%")
    return testing_acc, testing_loss

In [ ]:
# Initialize precision, recall, and F1 score metrics
precision = Precision(task="multiclass", num_classes=num_classes)
recall = Recall(task="multiclass", num_classes=num_classes)
f1 = F1Score(task="multiclass", num_classes=num_classes)
confusion_matrix = ConfusionMatrix(task="multiclass", num_classes=num_classes)

# Base Model Training

In [ ]:
# Set the seed and start the timer
torch.manual_seed(42)
torch.cuda.manual_seed(42)
train_start_time_on_gpu = timer()
# Set the number of epochs
epochs = 50
acc_TRAIN = []
acc_TEST = []
loss_TRAIN = []
loss_TEST = []
precision_result = []
recall_result = []
f1_result = []
# Training
for epoch in tqdm(range(epochs)):
    print(f"\nepoch: {epoch+1}\n--------------------------------------")
    ACC_TRAIN = 0
    LOSS_TRAIN = 0
    ACC_TEST = 0
    LOSS_TEST = 0
    # Training
    ACC_TRAIN, LOSS_TRAIN = train_step(model, cifar_train_data_loader, loss_fn, optimizer, accuracy_fn, device)
    # Testing
    ACC_TEST, LOSS_TEST = test_step(model, cifar_test_data_loader, loss_fn, accuracy_fn, device)
    # y_preds
    y_preds = []
    model.eval()
    with torch.inference_mode():
        for x, y in tqdm(cifar_test_data_loader, desc="Making Predictions..."):
            # Send the data to the target device
            x, y = x.to(device), y.to(device)
            # Forward Pass
            logits = model(x)
            # Pred probs then labels
            y_pred = torch.softmax(logits, dim=1).argmax(dim=1)
            # Put predictions on the cpu for evaluation
            y_preds.append(y_pred.cpu())
    # Concatenate the predicions of all batches
    y_pred_tensor = torch.cat(y_preds)
    # Update the metrics with true and predicted labels
    precision.update(y_pred_tensor, torch.tensor(cifar_test_data.targets))
    recall.update(y_pred_tensor, torch.tensor(cifar_test_data.targets))
    f1.update(y_pred_tensor, torch.tensor(cifar_test_data.targets))
    # List
    acc_TRAIN.append(ACC_TRAIN)
    acc_TEST.append(ACC_TEST)
    loss_TRAIN.append(LOSS_TRAIN.detach().cpu().numpy())
    loss_TEST.append(LOSS_TEST.detach().cpu().numpy())
    precision_result.append(precision.compute())
    recall_result.append(recall.compute())
    f1_result.append(f1.compute())
    
# Compute the time of the training
print("\nTraining is finished!\n")
train_end_time_on_gpu = timer()
display_train_time(
    train_start_time_on_gpu,
    train_end_time_on_gpu,
    device=next(model.parameters()).device,
)

In [ ]:
# Create a figure and axes
fig, ax = plt.subplots()
# Plot the data
plt.plot(acc_TRAIN, label="Accuracy in Training")
plt.plot(acc_TEST, label="Accuracy in Testing")
plt.plot(loss_TRAIN, label="Loss in Training")
plt.plot(loss_TEST, label="Loss in Testing")
plt.plot(precision_result, label="Precision")
plt.plot(recall_result, label="Recall")
plt.plot(f1_result, label="F1 Score")

# Add labels and title
plt.xlabel('Epochs')
plt.ylabel('Value')
plt.title('Evalution')
# Display the plot
plt.legend()
plt.show()
plt.savefig('reports/evals_base_model_before_al.png')
print(f"Precision: {precision_result[-1]}")
print(f"Recall: {recall_result[-1]}")
print(f"F1 Score: {f1_result[-1]}")

In [ ]:
torch.manual_seed(42)

In [ ]:
def eval_model(
    model: nn.Module,
    data_loader: torch.utils.data.DataLoader,
    loss_fn: nn.Module,
    accuracy_fn,
    device: torch.device = device,
):
    model.eval()
    loss, acc = 0, 0

    with torch.inference_mode():
        for x, y in tqdm(data_loader):
            x, y = x.to(device), y.to(device)
            # Forward Pass
            y_pred = model(x)
            # Loss
            loss += loss_fn(y_pred, y)
            acc += accuracy_fn(torch.argmax(y_pred, dim=1), y)
        loss /= len(data_loader)
        acc /= len(data_loader)
    return {
        "Model Name": model.__class__.__name__,
        "Model Loss": loss.item(),
        "Model Accuracy": acc,
    }, torch.argmax(y_pred, dim=1)

In [ ]:
# Testing the model
cifar_model_results, y_pred = eval_model(model, cifar_test_data_loader, loss_fn, accuracy_fn)

In [ ]:
cifar_model_results

In [ ]:
len(next(iter(cifar_test_data_loader)))

# Evaluating Model Visually


In [ ]:
test_samples = []
test_labels = []

for sample, label in random.sample(list(cifar_test_data), k=9):
    test_samples.append(sample)
    test_labels.append(label)

rows = 6
cols = 5

plt.figure(figsize=(13, 11))

model.eval()
with torch.inference_mode():
    for i in range(1, rows * cols + 1):
        plt.subplot(rows, cols, i)
        random_index = torch.randint(1, 9, size=[1]).item()
        image, label = test_samples[random_index], test_labels[random_index]
        pred_label = model(image.unsqueeze(dim=0).to(device)).argmax()
        image = image.permute(1, 2, 0).numpy()
        plt.imshow(image, interpolation="bilinear")
        if pred_label == label:
            plt.title(
                f"True: {cifar_class_names[label]} | Pred: {cifar_class_names[pred_label]}",
                c="g",
                fontsize=10,
            )
        else:
            plt.title(
                f"True: {cifar_class_names[label]} | Pred: {cifar_class_names[pred_label]}",
                c="r",
                fontsize=10,
            )
        plt.axis(False)
plt.savefig('reports/preds_base_model_before_al.png')

In [ ]:
confmat = ConfusionMatrix(task="multiclass", num_classes=len(cifar_class_names))
confmat_tensor = confmat(preds=y_pred_tensor, target=torch.tensor(cifar_test_data.targets))
fig, ax = plot_confusion_matrix(
    conf_mat=confmat_tensor.numpy(),
    class_names=cifar_class_names,
    figsize=(10, 7),
    show_absolute=True,
    show_normed=True,
    colorbar=True
)
plt.savefig('reports/conf_mat_base_model_before_al.png')

# Saving the Model


In [ ]:
# Create the models path
MODELS_PATH = Path("models")
MODELS_PATH.mkdir(parents=True, exist_ok=True)
# Create model save
MODEL_NAME = "base_model.pth"
MODEL_SAVE_PATH = MODELS_PATH / MODEL_NAME
# Save the model
print(f"Saving model to {MODEL_SAVE_PATH}")
torch.save(obj=model.state_dict(), f=MODEL_SAVE_PATH)

# Load the Model


In [ ]:
# Create a model instance
trained_model = models.resnet50().to(device)
trained_model.fc = nn.Linear(model.fc.in_features, 10)
trained_model.load_state_dict(torch.load(MODEL_SAVE_PATH))
trained_model.to(device)

In [ ]:
summary(trained_model, (3, 32, 32))

In [ ]:
trained_model = NeuralNetClassifier(
    trained_model,
    criterion=nn.CrossEntropyLoss,
    optimizer=torch.optim.Adam,
    train_split=None,
    verbose=1,
    device=device,
)

# Active Learning


In [ ]:
def active_learning(pool_x, pool_y, initial_x, initial_y, test_x, test_y, method, model, num_classes, class_names):
    learner = None
    cycles = 100
    acc = []
    precision_result = []
    recall_result = []
    f1_result = []
    if method == "uncertainty_sampling":
        learner = ActiveLearner(
            estimator=model,
            X_training=initial_x,
            y_training=initial_y,
            query_strategy=uncertainty_sampling,
        )
    elif method == "margin_sampling":
        learner = ActiveLearner(
            estimator=model,
            X_training=initial_x,
            y_training=initial_y,
            query_strategy=margin_sampling,
        )
    elif method == "entropy_sampling":
        learner = ActiveLearner(
            estimator=model,
            X_training=initial_x,
            y_training=initial_y,
            query_strategy=entropy_sampling,
        )
    elif method == "random_sampling":
        def random_sampling(Learner, pool_x, n_instances):
            n_samples = len(pool_x)
#             pool_x = pool_x.reshape(-1, 3, 32, 32)
            query_idx = np.random.choice(range(n_samples), size=n_instances, replace=False)
            X_query = pool_x[query_idx]
            return query_idx, X_query
#         initial_x = initial_x.reshape(-1, 3, 32, 32)
        initial_x = initial_x.astype(np.float32)
        learner = ActiveLearner(
            estimator=model,
            X_training=initial_x,
            y_training=initial_y,
            query_strategy=random_sampling,
        )
    else:
        print('Invalid Input')
    print(f"Starting Active Learning with {method} method")
    for cycle in range(cycles):
        print(f"\nCycle: {cycle+1}")
        query_idx, query_instance = learner.query(pool_x, n_instances=1)#n_instances=100
#         pool_x = pool_x.reshape(-1, 3, 32, 32)
        learner.teach(X=pool_x[query_idx], y=pool_y[query_idx])
        # remove queried instance from pool_x
        pool_x = np.delete(pool_x, query_idx, axis=0)
        pool_y = np.delete(pool_y, query_idx, axis=0)
        model_accuracy = learner.score(np.concatenate((initial_x, pool_x), axis=0), np.concatenate((initial_y, pool_y), axis=0))
        print(f'Accuracy after query {cycle+1}: {model_accuracy:0.4f}')
        acc.append(model_accuracy)
        
        y_pred = learner.predict(test_x)
        # Initialize precision, recall, and F1 score metrics
        precision = Precision(task="multiclass", num_classes=num_classes)
        recall = Recall(task="multiclass", num_classes=num_classes)
        f1 = F1Score(task="multiclass", num_classes=num_classes)
        # Update the metrics with true and predicted labels
        precision.update(torch.tensor(y_pred), torch.tensor(test_y))
        recall.update(torch.tensor(y_pred), torch.tensor(test_y))
        f1.update(torch.tensor(y_pred), torch.tensor(test_y))
        # Compute the metrics
        precision_result.append(precision.compute())
        recall_result.append(recall.compute())
        f1_result.append(f1.compute())
        
    confusion_matrix = ConfusionMatrix(task="multiclass", num_classes=num_classes)
    confusion_matrix.update(torch.tensor(y_pred), torch.tensor(test_y))
    confusion_matrix.compute()
    fig, ax = plot_confusion_matrix(conf_mat=confusion_matrix.compute().numpy(), class_names=class_names, figsize=(10, 7), show_absolute=True, show_normed=True, colorbar=True)
    plt.savefig(f'reports/conf_mat_{method}_after_al.png')
    return acc, precision_result, recall_result, f1_result

In [ ]:
x_pool = x_pool.reshape(-1, 3, 32, 32)
x_initial = x_initial.reshape(-1, 3, 32, 32)
x_test = x_test.reshape(-1, 3, 32, 32)

x_pool = x_pool.astype(np.float32)
x_initial = x_initial.astype(np.float32)
x_test = x_test.astype(np.float32)

In [ ]:
x_pool.shape, x_initial.shape, x_test.shape

In [ ]:
acc_random_sampling, precision_result_random_sampling, recall_result_random_sampling, f1_result_random_sampling = active_learning(
    x_pool, y_pool, x_initial, y_initial, x_test, y_test, "random_sampling", trained_model, 10, cifar_class_names
)

In [ ]:
acc_uncertainty_sampling, precision_result_uncertainty_sampling, recall_result_uncertainty_sampling, f1_result_uncertainty_sampling = active_learning(
    x_pool, y_pool, x_initial, y_initial, x_test, y_test, "uncertainty_sampling", trained_model, 10, cifar_class_names
)

In [ ]:
acc_margin_sampling, precision_result_margin_sampling, recall_result_margin_sampling, f1_result_margin_sampling = active_learning(
    x_pool, y_pool, x_initial, y_initial, x_test, y_test, "margin_sampling", trained_model, 10, cifar_class_names
)

In [ ]:
acc_entropy_sampling, precision_result_entropy_sampling, recall_result_entropy_sampling, f1_result_entropy_sampling = active_learning(
    x_pool, y_pool, x_initial, y_initial, x_test, y_test, "entropy_sampling", trained_model, 10, cifar_class_names
)

# Plot All Metrics

# Accuracy

In [ ]:
# Create a figure and axes
fig, ax = plt.subplots()
# Plot the data
plt.plot(acc_random_sampling, label="acc_random_sampling")
plt.plot(acc_uncertainty_sampling, label="acc_uncertainty_sampling")
plt.plot(acc_margin_sampling, label="acc_margin_sampling")
plt.plot(acc_entropy_sampling, label="acc_entropy_sampling")

# Add labels and title
plt.xlabel('Epochs')
plt.ylabel('Value')
plt.title('Accuracy')
# Display the plot
plt.legend()
plt.show()
plt.savefig('reports/acc_base_model_after_al.png')

print(f"acc_random_sampling: {acc_random_sampling[-1]}")
print(f"acc_uncertainty_sampling: {acc_uncertainty_sampling[-1]}")
print(f"acc_margin_sampling: {acc_margin_sampling[-1]}")
print(f"acc_entropy_sampling: {acc_entropy_sampling[-1]}")

# Precision

In [ ]:
# Create a figure and axes
fig, ax = plt.subplots()
# Plot the data
plt.plot(precision_result_random_sampling, label="precision_result_random_sampling")
plt.plot(precision_result_uncertainty_sampling, label="precision_result_uncertainty_sampling")
plt.plot(precision_result_margin_sampling, label="precision_result_margin_sampling")
plt.plot(precision_result_entropy_sampling, label="precision_result_entropy_sampling")

# Add labels and title
plt.xlabel('Epochs')
plt.ylabel('Value')
plt.title('Precision')
# Display the plot
plt.legend()
plt.show()
plt.savefig('reports/prec_base_model_after_al.png')

print(f"precision_result_random_sampling: {precision_result_random_sampling[-1]}")
print(f"precision_result_uncertainty_sampling: {precision_result_uncertainty_sampling[-1]}")
print(f"precision_result_margin_sampling: {precision_result_margin_sampling[-1]}")
print(f"precision_result_entropy_sampling: {precision_result_entropy_sampling[-1]}")

# Recall

In [ ]:
# Create a figure and axes
fig, ax = plt.subplots()
# Plot the data
plt.plot(recall_result_random_sampling, label="recall_result_random_sampling")
plt.plot(recall_result_uncertainty_sampling, label="recall_result_uncertainty_sampling")
plt.plot(recall_result_margin_sampling, label="recall_result_margin_sampling")
plt.plot(recall_result_entropy_sampling, label="recall_result_entropy_sampling")

# Add labels and title
plt.xlabel('Epochs')
plt.ylabel('Value')
plt.title('Recall')
# Display the plot
plt.legend()
plt.show()
plt.savefig('reports/recall_base_model_after_al.png')

print(f"recall_result_random_sampling: {recall_result_random_sampling[-1]}")
print(f"recall_result_uncertainty_sampling: {recall_result_uncertainty_sampling[-1]}")
print(f"recall_result_margin_sampling: {recall_result_margin_sampling[-1]}")
print(f"recall_result_entropy_sampling: {recall_result_entropy_sampling[-1]}")

# F1 Score

In [ ]:
# Create a figure and axes
fig, ax = plt.subplots()
# Plot the data
plt.plot(f1_result_random_sampling, label="f1_result_random_sampling")
plt.plot(f1_result_uncertainty_sampling, label="f1_result_uncertainty_sampling")
plt.plot(f1_result_margin_sampling, label="f1_result_margin_sampling")
plt.plot(f1_result_entropy_sampling, label="f1_result_entropy_sampling")

# Add labels and title
plt.xlabel('Epochs')
plt.ylabel('Value')
plt.title('F1 Score')
# Display the plot
plt.legend()
plt.show()
plt.savefig('reports/f1_base_model_after_al.png')

print(f"f1_result_random_sampling: {f1_result_random_sampling[-1]}")
print(f"f1_result_uncertainty_sampling: {f1_result_uncertainty_sampling[-1]}")
print(f"f1_result_margin_sampling: {f1_result_margin_sampling[-1]}")
print(f"f1_result_entropy_sampling: {f1_result_entropy_sampling[-1]}")